<a href="https://colab.research.google.com/github/alirezash97/Convolutional-Neural-Network/blob/master/Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def relu(x):
  return max(0, x)

In [0]:
def convolution(image, Cnn_filter):
  x = Cnn_filter.shape[0]
  y = Cnn_filter.shape[1]
  out = np.zeros(((int(image.shape[0] - x + 1)), (int(image.shape[1] - y + 1))))
  for i in range(out.shape[0]):
    for j in range(out.shape[1]):
      temp = np.sum(image[i:i+x, j:j+y]*Cnn_filter)
      out[i][j] = relu(temp) 


  return out

In [0]:
def pooling(filtered, window):
  x = window[0]
  y = window[1]
  out = np.zeros((int(filtered.shape[0] / x), int(filtered.shape[1] / y)))
  for i in range(out.shape[0]):
    for j in range(out.shape[1]):
      temp = np.array(filtered[i*x:(i+1)*x, j*x:(j+1)*x])
      out[i][j] = np.max(temp)
  
  return out


In [0]:
def convolution_layer(image, number_of_outputs, filter_shape, maxpool_window):
  

  weights = np.random.randint(low = -7, high = 7, size=(number_of_outputs, filter_shape[0], filter_shape[1], 3 ))
  p_input = image.shape[0]
  p_output = int((image.shape[0] - (filter_shape[0] - 1)) / maxpool_window[0]) 
  output = np.zeros((p_output, p_output, 3, number_of_outputs))
  
  for i in tqdm(range(number_of_outputs)):
    filter_1 = weights[i, :, :, 0]
    filter_2 = weights[i, :, :, 1]
    filter_3 = weights[i, :, :, 2]

    b = image[:p_input, :p_input, :1]
    c = image[:p_input, :p_input, 1:2]
    d = image[:p_input, :p_input, 2:3]

    out1 = convolution(b, filter_1)
    out2 = convolution(c, filter_2)
    out3 = convolution(d, filter_3)

    outt1 = pooling(out1, maxpool_window)
    outt2 = pooling(out2, maxpool_window)
    outt3 = pooling(out3, maxpool_window)

    filtered_image = np.zeros((p_output, p_output, 3))
    filtered_image[:p_output, :p_output, :1] = outt1.reshape((p_output, p_output, 1))
    filtered_image[:p_output, :p_output, 1:2] = outt2.reshape((p_output, p_output, 1))
    filtered_image[:p_output, :p_output, 2:3] = outt3.reshape((p_output, p_output, 1))

    output[:, :, :, i] = filtered_image


  return output

In [0]:
a = train_images[1972] / 255.
b = 10
c = [3, 3]
d = [2, 2]
my_images = convolution_layer(a, b, c, d)
print("\n",my_images.shape)

In [0]:
def initialization(image, layers):


  w = []
  b = []

  # flatten_image = image.flatten()
  weight1 = np.ones((image.shape[1], layers[0]))
  w.append(weight1)
  for i in range (len(layers)-1):

    temp_weight = np.ones((layers[i], layers[i+1]))
    w.append(temp_weight)
    temp_bias = np.ones((layers[i], 1))
    b.append(temp_bias)
  temp_bias = np.ones((layers[-1], 1))
  b.append(temp_bias)

  w = np.array(w)
  b = np.array(b)
  
  return w, b

In [0]:
def feedforward(image, weights, bias):
  
  z = []
  a = []
  
  # flatten_image = image.flatten()
  x = np.array(image) # image.reshape(len(flatten_image), 1))
  z_temp = np.dot(x, weights[0]).T + bias[0]
  z.append(z_temp)
  
  a_temp = sigmoid(z_temp)
  a.append(a_temp)

  for i in range(len(weights)-1):
    
    z_temp = np.dot(a[i].T, weights[i+1]).T + bias[i+1]
    z.append(z_temp)
    
    a_temp = sigmoid(z_temp)
    a.append(a_temp)

    
    # print(a[i].T)
    # print(weights[i+1].shape)
    # print(z_temp.shape)
  
  z = np.array(z)
  a = np.array(a)

  return z, a
  

In [0]:
def sigmoid(z):

  a = 1 / (1 + np.exp((-1*z)))

  return a

In [0]:
def backpropagation(predicted, actual, z, a, w, b, x):

  dz = []
  dw = []
  db = []
  m = predicted.shape[0]
  dz_temp = a[-1].T - actual
  dz_temp = dz_temp.T
  dz.append(dz_temp)
  for i in reversed(range(len(a)-1)):
    # print(dz_temp)
    # print(i)
    # print(a[-1])
    dw_temp = np.dot(dz_temp, a[i].T) / m
    dw.append(dw_temp)
    db_temp = np.sum(dz_temp, axis = 1, keepdims=True) / m
    db.append(db_temp)
    dz_temp = np.dot(w[i+1], dz_temp) * sigmoid_derivate(z[i])
    dz.append(dz_temp)
    

  dw_temp = np.dot(dz_temp, x) / m
  dw.append(dw_temp)
  db_temp = np.sum(dz_temp, axis = 1, keepdims=True) / m
  db.append(db_temp)




  return dw, db


In [0]:
def sigmoid_derivate(z):
  
  
  a = z*(1-z)
  return a

In [0]:
xx_xx = np.array([[3, 4, 0, 1, 5], #flatten image 1 (consist of so many filters)
              [0, 0, 0, 0, 0], #flatten image 2 (consist of so many filters)
              [0, 6, 4, 2, 3],
              [1, 7, 4, 1, 8]])
y = np.array([[0.1, 0.82],
              [0.23, 0],
              [0.5, 1],
              [0.75, 0]])

# w, b = initialization(x, [10, 3, 6, 10, 1])
# z, a = feedforward(x, w, b)


# x = my_images[:, :, :, :] 
ww_ww, bb_bb = initialization(x, [10, 3, 6, 10, 2])
zz_zz, aa_aa = feedforward(x, w, b)
dw, db = backpropagation(aa_aa[-1], y, zz_zz, aa_aa, ww_ww, bb_bb, xx_xx)

# print(x.T)
# print(z[0].T)


print(dw[4].T)
# print(db[4].shape)
print(w[0])
# print(b[0].shape)
# print(z[1].T)
# print(w[2])
# print(a[2])
# printa[-1].T)
# print(z[1])
# print(a[1])